In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\kidney_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\kidney_disease_classification'

In [5]:
from dataclasses import dataclass, field
from pathlib import Path
from box import Box
from typing import Dict, Any


@dataclass(frozen=True)
class TrainingConfig:
    """
    Configuration for the model training stage.
    """
    root_dir: Path  # Root directory for training artifacts
    trained_model_path: Path  # Path to save the final trained model
    updated_base_model_path: Path  # Path to the base model from the preparation stage
    training_data: Path  # Path to the training dataset
    validation_data: Path # Path to the validation dataset
    params_epochs: int  # Number of training epochs
    params_batch_size: int  # Number of samples per batch
    params_is_augmentation: bool  # Flag to enable/disable data augmentation
    params_image_size: list  # Image dimensions for training
    params_learning_rate: float  # Learning rate for the optimizer
    params_callbacks: Dict[str, Any]  # Dictionary of callbacks (e.g., EarlyStopping, ReduceLROnPlateau)

In [6]:
from KidneyCNN.constants import *
from KidneyCNN.utils.common import read_yaml, create_directories
import tensorflow as tf
import os
from pathlib import Path
# Import the Keras callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
            """
            Provides the configuration for the model training stage.
            """
            # Get settings from main config and params files
            training = self.config.training
            prepare_base_model = self.config.prepare_base_model
            params = self.params
            
            # Define the path to the training data directory
            training_data = os.path.join(self.config.data_transformation.data_split_dir, "train")
            # Define the path to the validation data directory
            validation_data = os.path.join(self.config.data_transformation.data_split_dir,  "val")
            
            # Create the training artifacts directory
            create_directories([Path(training.root_dir)])

            # Instantiate the callbacks here using parameters from params.yaml
            callbacks = {
                "early_stopping": EarlyStopping(
                    monitor=params.CALLBACKS.early_stopping.monitor,
                    patience=params.CALLBACKS.early_stopping.patience,
                    verbose=params.CALLBACKS.early_stopping.verbose
                ),
                "model_checkpoint": ModelCheckpoint(
                    filepath=Path(training.trained_model_path),
                    save_best_only=params.CALLBACKS.model_checkpoint.save_best_only,
                    monitor=params.CALLBACKS.model_checkpoint.monitor
                ),
                "reduce_lr_on_plateau": ReduceLROnPlateau(
                    monitor=params.CALLBACKS.reduce_lr_on_plateau.monitor,
                    factor=params.CALLBACKS.reduce_lr_on_plateau.factor,
                    patience=params.CALLBACKS.reduce_lr_on_plateau.patience
                )
            }
            
            # Create a TrainingConfig object with all necessary parameters
            training_config = TrainingConfig(
                root_dir=Path(training.root_dir),
                trained_model_path=Path(training.trained_model_path),
                updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
                training_data=Path(training_data),
                validation_data=Path(validation_data),
                params_epochs=params.EPOCHS, # Number of training epochs
                params_batch_size=params.BATCH_SIZE, # Batch size for training
                params_is_augmentation=params.AUGMENTATION,  # Enable/disable data augmentation
                params_image_size=params.IMAGE_SIZE,  # Set the input image size
                params_learning_rate=params.LEARNING_RATE,  # Define the learning rate
                # Pass the correctly instantiated callback objects to the dataclass
                params_callbacks=callbacks
            )

            return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from pathlib import Path

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    # This method is for compiling the model with an optimizer and loss function.
    def compile_model(self):
        # The .compile() method configures the model for training.
        self.model.compile(
            # The optimizer updates the model based on the data and the loss function.
            # Here, we use Stochastic Gradient Descent (SGD) with the learning rate from the config.
            optimizer = tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            # The loss function measures how accurate the model is.
            # CategoricalCrossentropy is suitable for multi-class classification problems.
            loss = tf.keras.losses.CategoricalCrossentropy(),
            # The metrics list is used to monitor the training and testing steps.
            # "accuracy" is a common metric to track model performance.
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        # This method prepares and configures data generators for training and validation.
        # Data generators load images from disk and apply transformations on the fly,
        # which is memory-efficient for large datasets.

        # Arguments for the ImageDataGenerator.
        # `rescale` normalizes pixel values to the range [0, 1].
        # `validation_split` reserves a portion of the data for validation.
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        # Arguments for the .flow_from_directory() method.
        # `target_size` resizes all images to the specified dimensions.
        # `batch_size` is the number of images to be yielded from the generator.
        # `interpolation` defines the method for resizing images.
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        # Create an ImageDataGenerator instance with the defined arguments.
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Verify that the training and validation data directories exist before proceeding.
        if not os.path.exists(self.config.training_data):
            raise FileNotFoundError(f"Training data directory not found at: {self.config.training_data}")
        
        if not os.path.exists(self.config.validation_data):
            raise FileNotFoundError(f"Validation data directory not found at: {self.config.validation_data}")


        # The training data is assumed to be in the 'train' folder
        # The validation data is assumed to be in the 'valid' folder
        self.train_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # Correctly pointing to the separate validation data directory
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        # Load the base model first
        self.get_base_model()
        # Compile the model with an optimizer, loss, and metrics
        self.compile_model()
        # Prepare the data generators
        self.train_valid_generator()
        
        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            callbacks=list(self.config.params_callbacks.values())
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    # Create an instance of the ConfigurationManager to get configurations
    config = ConfigurationManager()
    
    # Get the specific configuration for the 'Training' component
    training_config = config.get_training_config()
    
    # Instantiate the Training component with the fetched configuration
    training = Training(config=training_config)
    
    # 1. Load the model's architecture and weights. This model has been
    #    prepared in a previous stage (PrepareBaseModel).
    training.get_base_model() 
    
    # 2. Re-compile the model. This is the crucial step because the optimizer's
    #    state is not saved with the model architecture. Compiling here
    #    ensures the model is ready for training.
    training.compile_model() 
    
    # 3. Set up the data generators. This handles data loading, preprocessing,
    #    and augmentation for both the training and validation sets.
    training.train_valid_generator()
    
    # 4. Start the training process. The model will now be trained on the
    #    prepared data using the specified epochs, batch size, and callbacks.
    training.train()
    
except Exception as e:
    raise e

[2025-10-04 23:47:08,648: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-04 23:47:08,656: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-04 23:47:08,659: INFO: common: created directory at: artifacts]
[2025-10-04 23:47:08,659: INFO: common: created directory at: artifacts\training]
Found 3360 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Found 3360 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
210/210 [==============================] - 748s 4s/step - loss: 14.2374 - accuracy: 0.3902 - val_loss: 7.9576 - val_accuracy: 0.4500 - lr: 0.0100
